In [1]:
!pip install --upgrade pip
!pip install transformers==4.43.3 datasets==3.0.1 torch torchvision pillow pandas tqdm accelerate evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 106.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 22.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 61.2 MB/s  0:00:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 39.6 MB/s  0:00:12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 28.1 MB/s  0:00:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 54.5 MB/s  0:00:01
   ━━━━━━━━━

In [2]:
import os
import json
import pandas as pd
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm
import torch

from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from torch.nn.utils.rnn import pad_sequence
from datasets import Dataset

# Paths
INPUT_DIR = "/kaggle/input/01-paddle-detector"
JSON_PATH = os.path.join(INPUT_DIR, "results/boxes.json")
CROP_DIR = os.path.join(INPUT_DIR, "results/crops")
OUTPUT_DIR = "/kaggle/working/trocr_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Device set: {device}")
print("✅ Paths initialized")


✅ Device set: cuda
✅ Paths initialized


In [3]:
with open(JSON_PATH, "r") as f:
    metadata = json.load(f)

print(f"✅ Loaded metadata: {len(metadata)} entries")


✅ Loaded metadata: 103 entries


In [4]:
MODEL_NAME = "microsoft/trocr-base-printed"

processor = TrOCRProcessor.from_pretrained(MODEL_NAME)
model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME).to(device)

# Config fixes
if model.config.decoder_start_token_id is None:
    model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
if model.config.pad_token_id is None:
    model.config.pad_token_id = processor.tokenizer.pad_token_id
if model.config.eos_token_id is None:
    model.config.eos_token_id = processor.tokenizer.eos_token_id

print(f"✅ TrOCR loaded | decoder_start_token_id={model.config.decoder_start_token_id}, pad_token_id={model.config.pad_token_id}")


2025-10-20 10:58:03.117780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760957883.280787      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760957883.329289      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ TrOCR loaded | decoder_start_token_id=0, pad_token_id=1


In [5]:
def load_image_safe(path):
    try:
        return Image.open(path).convert("RGB")
    except (UnidentifiedImageError, OSError) as e:
        print(f"⚠️ Skipping corrupted image: {path}")
        return None


In [6]:
# Convert metadata into pandas DataFrame
def load_training_data(metadata, crop_dir):
    samples = []
    for entry in metadata:
        crop_path = os.path.join(crop_dir, f"{entry['id']}.jpg")
        if os.path.exists(crop_path):
            samples.append({"image_path": crop_path, "text": entry["paddle_text"]})
    return pd.DataFrame(samples)

train_df = load_training_data(metadata, CROP_DIR)
print(f"✅ Total samples before filtering: {len(train_df)}")

# Filter out corrupted images
valid_samples = []
for idx, row in train_df.iterrows():
    if load_image_safe(row["image_path"]) is not None:
        valid_samples.append(row)
train_df = pd.DataFrame(valid_samples)
print(f"✅ Total valid samples: {len(train_df)}")

# Create HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df)


✅ Total samples before filtering: 103
✅ Total valid samples: 103


In [7]:
def preprocess_function(examples):
    images = []
    labels = []
    for img_path, txt in zip(examples["image_path"], examples["text"]):
        image = load_image_safe(img_path)
        if image is None:
            continue
        images.append(image)
        labels.append(txt)
    
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    input_ids = processor.tokenizer(labels, padding="max_length", truncation=True, return_tensors="pt").input_ids
    return {"pixel_values": pixel_values, "labels": input_ids}

train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
print("✅ Dataset preprocessed")


Map:   0%|          | 0/103 [00:00<?, ? examples/s]

✅ Dataset preprocessed


In [8]:
def collate_fn(batch):
    # batch is a list of dicts
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = [item["labels"].squeeze(0) for item in batch]
    labels = pad_sequence(labels, batch_first=True, padding_value=model.config.pad_token_id)
    return {"pixel_values": pixel_values.to(device), "labels": labels.to(device)}


In [9]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)


✅ Device: cuda


In [10]:
from torch.utils.data import Dataset
from PIL import Image

class TrOCRDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.samples = dataframe.to_dict('records')
        self.processor = processor

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        image = Image.open(sample['image_path']).convert("RGB")
        pixel_values = self.processor(images=image, return_tensors="pt").pixel_values.squeeze(0)
        labels = torch.tensor(self.processor.tokenizer(sample['text'], 
                                                       padding="max_length",
                                                       truncation=True,
                                                       return_tensors="pt").input_ids.squeeze(0))
        return {"pixel_values": pixel_values, "labels": labels}

# Wrap in dataset
torch_dataset = TrOCRDataset(train_df, processor)
print("✅ Dataset ready with", len(torch_dataset), "samples")


✅ Dataset ready with 103 samples


In [11]:
def collate_fn(batch):
    pixel_values = torch.stack([x["pixel_values"] for x in batch]).to(device)
    labels = [x["labels"] for x in batch]
    labels = pad_sequence(labels, batch_first=True, padding_value=model.config.pad_token_id).to(device)
    return {"pixel_values": pixel_values, "labels": labels}

train_loader = DataLoader(torch_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
print("✅ DataLoader ready")


✅ DataLoader ready


In [12]:
optimizer = AdamW(model.parameters(), lr=5e-5)
scaler = torch.amp.GradScaler(device="cuda")  # FP16 mixed precision

In [13]:
num_epochs = 3
gradient_accumulation_steps = 4
save_dir = "/kaggle/working/trocr_finetuned"
os.makedirs(save_dir, exist_ok=True)

model.train()
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = torch.amp.GradScaler(device="cuda")  # Updated to remove warning

global_step = 0

for epoch in range(num_epochs):
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    optimizer.zero_grad()
    
    for step, batch in enumerate(loop):
        # Move tensors to GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        
        with torch.amp.autocast(device_type="cuda"):  # FP16
            outputs = model(pixel_values=batch["pixel_values"], labels=batch["labels"])
            loss = outputs.loss / gradient_accumulation_steps
        
        scaler.scale(loss).backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1

            if global_step % 50 == 0:
                loop.set_postfix(loss=loss.item()*gradient_accumulation_steps)

    # Save checkpoint at end of epoch
    epoch_save_path = os.path.join(save_dir, f"checkpoint-epoch{epoch+1}")
    model.save_pretrained(epoch_save_path)
    processor.save_pretrained(epoch_save_path)
    print(f"✅ Epoch {epoch+1} checkpoint saved to {epoch_save_path}")

print("✅ Training complete")


Epoch 1/3:   0%|          | 0/52 [00:00<?, ?it/s]/tmp/ipykernel_19/702183868.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.processor.tokenizer(sample['text'],
Epoch 1/3: 100%|██████████| 52/52 [00:17<00:00,  3.03it/s]


✅ Epoch 1 checkpoint saved to /kaggle/working/trocr_finetuned/checkpoint-epoch1


Epoch 2/3:   0%|          | 0/52 [00:00<?, ?it/s]/tmp/ipykernel_19/702183868.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.processor.tokenizer(sample['text'],
Epoch 2/3: 100%|██████████| 52/52 [00:17<00:00,  3.05it/s]


✅ Epoch 2 checkpoint saved to /kaggle/working/trocr_finetuned/checkpoint-epoch2


Epoch 3/3:   0%|          | 0/52 [00:00<?, ?it/s]/tmp/ipykernel_19/702183868.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.processor.tokenizer(sample['text'],
Epoch 3/3: 100%|██████████| 52/52 [00:17<00:00,  2.99it/s]


✅ Epoch 3 checkpoint saved to /kaggle/working/trocr_finetuned/checkpoint-epoch3
✅ Training complete


In [14]:
import os
import json
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

# --- Paths ---
INPUT_DIR = "/kaggle/input/01-paddle-detector"
JSON_PATH = os.path.join(INPUT_DIR, "results/boxes.json")
CROP_DIR = os.path.join(INPUT_DIR, "results/crops")
OUTPUT_DIR = "/kaggle/working/trocr_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Device ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# --- Load TrOCR ---
MODEL_NAME = "microsoft/trocr-base-printed"
processor = TrOCRProcessor.from_pretrained(MODEL_NAME)
model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

# --- Load boxes (optional, just for reference) ---
with open(JSON_PATH, "r") as f:
    boxes = json.load(f)

# --- Inference ---
results = []

for filename in tqdm(sorted(os.listdir(CROP_DIR)), desc="Running TrOCR"):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    image_path = os.path.join(CROP_DIR, filename)
    try:
        image = Image.open(image_path).convert("RGB")
    except UnidentifiedImageError:
        print(f"⚠️ Could not open image: {filename}")
        continue

    # Convert image to model input
    pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)

    # Generate text (increased max_new_tokens, beam search)
    generated_ids = model.generate(
        pixel_values=pixel_values,
        max_new_tokens=50,  # allows longer predictions
        num_beams=5,
        early_stopping=True
    )

    text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    results.append({
        "crop_filename": filename,
        "trocr_text": text,
        "confidence": "N/A"  # TrOCR base model does not return confidence
    })

# --- Save results ---
import pandas as pd
df = pd.DataFrame(results)
df.to_csv(os.path.join(OUTPUT_DIR, "trocr_results.csv"), index=False)

with open(os.path.join(OUTPUT_DIR, "trocr_results.json"), "w") as f:
    json.dump(results, f, indent=2)

print(f"✅ TrOCR inference complete, {len(results)} results saved to {OUTPUT_DIR}")


✅ Using device: cuda


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Running TrOCR: 100%|██████████| 103/103 [00:52<00:00,  1.96it/s]

✅ TrOCR inference complete, 103 results saved to /kaggle/working/trocr_results


In [15]:
import random
from IPython.display import display
from PIL import Image

# Load results
results_csv = os.path.join(OUTPUT_DIR, "trocr_results.csv")
df = pd.read_csv(results_csv)

# Pick a few random samples to display
num_samples = 5
samples = df.sample(num_samples, random_state=42)

for _, row in samples.iterrows():
    crop_path = os.path.join(CROP_DIR, row["crop_filename"])
    try:
        img = Image.open(crop_path).convert("RGB")
        display(img)
    except Exception as e:
        print(f"⚠️ Could not open image {row['crop_filename']}: {e}")
    
    print(f"Crop: {row['crop_filename']}")
    print(f"TrOCR Text: {row['trocr_text']}")
    print(f"Confidence: {row['confidence']}")
    print("-" * 50)


Crop: 0_36.jpg
TrOCR Text: 13:56:52
Confidence: nan
--------------------------------------------------


Crop: 1_22.jpg
TrOCR Text: ELM, SEH DIAM MORUMMY MIDN MOO ULT UT MORGET
Confidence: nan
--------------------------------------------------


Crop: 1_18.jpg
TrOCR Text: LOREM IPSUM DOLOR SIT AMEL, CONNECTUR ADJDISING
Confidence: nan
--------------------------------------------------


Crop: 0_51.jpg
TrOCR Text: SCANNED WITH CAMSCANNER
Confidence: nan
--------------------------------------------------


Crop: 0_47.jpg
TrOCR Text: FOR REFUND OR EXCHANGE WITH RECEIPT
Confidence: nan
--------------------------------------------------
